In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2019-Dec.csv
/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2019-Nov.csv
/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2019-Oct.csv
/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2020-Feb.csv
/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2020-Jan.csv


In [2]:
cos = pd.read_csv("/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2019-Dec.csv")

Creating a Recommender System using eCommerce behavior data frommulti-category cosmetic shop. The Data is five months of shopping but for the sake oflack of computing power we only used one month (December). Dataset contains 9 columns and 1.6 million unique values. Each row in the file represents an event. The events are all related to products and users. Each event is like many-to-many relationbetween products and users.

In [3]:
cos.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


Load the data

In [4]:
cos.describe()

,product_id,category_id,price,user_id
count,3.533286e+06,3.533286e+06,3.533286e+06,3.533286e+06
mean,5.473054e+06,1.555023e+18,8.871856e+00,5.223318e+08
std,1.331331e+06,1.689262e+17,1.986474e+01,8.494819e+07
min,3.752000e+03,1.487580e+18,-7.937000e+01,1.180452e+06
25%,5.726191e+06,1.487580e+18,2.060000e+00,4.866830e+08
50%,5.811429e+06,1.487580e+18,4.210000e+00,5.566496e+08
75%,5.859462e+06,1.487580e+18,7.140000e+00,5.828019e+08
max,5.917178e+06,2.235524e+18,3.277800e+02,5.954145e+08


Looking quickly at some statistics of it

In [5]:
cos["event_type"].value_counts()


view                1728331
cart                 927124
remove_from_cart     664655
purchase             213176
Name: event_type, dtype: int64

Examining different cart events but for the scope of this project we are only looking at "purchases" which has 56,117 cases

In [6]:
cos["product_id"].value_counts()


5809910    35225
5809912    10875
5909810     9671
5700037     8834
5802432     8411
           ...  
5777688        1
5700950        1
5839166        1
5917012        1
5557728        1
Name: product_id, Length: 44624, dtype: int64

In [7]:
cos["event_type"] = cos["event_type"].astype('category')
cos.dtypes

event_time         object
event_type       category
product_id          int64
category_id         int64
category_code      object
brand              object
price             float64
user_id             int64
user_session       object
dtype: object

In [8]:
cosdummy = pd.get_dummies(cos[['event_type']])
cosdummy['product_id'] = cos['product_id']
cosdummy['category_id'] = cos['category_id']
cosdummy['brand'] = cos['brand']
cosdummy['price'] = cos['price']
cosdummy['user_id'] = cos['user_id']
cosdummy

,event_type_cart,event_type_purchase,event_type_remove_from_cart,event_type_view,product_id,category_id,brand,price,user_id
0,0,0,1,0,5712790,1487580005268456287,f.o.x,6.27,576802932
1,0,0,0,1,5764655,1487580005411062629,cnd,29.05,412120092
2,1,0,0,0,4958,1487580009471148064,runail,1.19,494077766
3,0,0,0,1,5848413,1487580007675986893,freedecor,0.79,348405118
4,0,0,0,1,5824148,1487580005511725929,NaN,5.56,576005683
...,...,...,...,...,...,...,...,...,...
3533281,0,0,0,1,5683350,1487580005671109489,masura,2.84,536812729
3533282,0,0,0,1,5888097,1487580013388628160,shik,179.05,503658154
3533283,0,0,0,1,59975,1487580012096782476,NaN,7.14,595414541
3533284,0,0,0,1,5775982,1783999063314661546,NaN,11.90,397780878


Converting all the event types into binary variables

In [9]:
cosmetic = cosdummy.drop(['brand','category_id','event_type_view','event_type_remove_from_cart','event_type_cart','price'], axis = 1)
cosmetic

,event_type_purchase,product_id,user_id
0,0,5712790,576802932
1,0,5764655,412120092
2,0,4958,494077766
3,0,5848413,348405118
4,0,5824148,576005683
...,...,...,...
3533281,0,5683350,536812729
3533282,0,5888097,503658154
3533283,0,59975,595414541
3533284,0,5775982,397780878


In [11]:
cosmetic = cosmetic.loc[0:500]

table = pd.pivot_table(cosmetic, values='event_type_purchase', index=['user_id'],
                    columns=['product_id'] , aggfunc= sum, fill_value=0)
table

product_id,3936,3978,4497,4554,4600,4607,4640,4645,4689,4751,...,5908210,5908214,5908217,5908219,5908228,5908229,5908236,5908239,5908746,5908747
user_id,,,,,,,,,,,,,,,,,,,,,
247879837,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
252311454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
348405118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
361564578,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
407458156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579971574,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
579971759,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
579972058,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#Computer is not powerful enough to run the entire file so I used a subsection of it for analysis
cosmetic1 = cosdummy[:500]

In [13]:
cosmetic1
cosmetic1["brand"] = cosmetic1["brand"].astype('category')
cosmetic1.dtypes

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


event_type_cart                   uint8
event_type_purchase               uint8
event_type_remove_from_cart       uint8
event_type_view                   uint8
product_id                        int64
category_id                       int64
brand                          category
price                           float64
user_id                           int64
dtype: object

In [14]:
cosmetic1


,event_type_cart,event_type_purchase,event_type_remove_from_cart,event_type_view,product_id,category_id,brand,price,user_id
0,0,0,1,0,5712790,1487580005268456287,f.o.x,6.27,576802932
1,0,0,0,1,5764655,1487580005411062629,cnd,29.05,412120092
2,1,0,0,0,4958,1487580009471148064,runail,1.19,494077766
3,0,0,0,1,5848413,1487580007675986893,freedecor,0.79,348405118
4,0,0,0,1,5824148,1487580005511725929,NaN,5.56,576005683
...,...,...,...,...,...,...,...,...,...
495,0,0,0,1,5885416,1487580005092295511,grattol,6.08,492321842
496,0,0,0,1,5785144,1487580013950664926,runail,11.83,579950721
497,0,0,0,1,5550162,1487580010125459512,orly,11.33,412120092
498,1,0,0,0,5858911,1783999071199952917,NaN,0.79,473613801


In [15]:
#grouping by the user and product ID to create a ratings dataframe for future usage
ratings = cosmetic1.groupby(['user_id', 'product_id']).sum().reset_index()
ratings

,user_id,product_id,event_type_cart,event_type_purchase,event_type_remove_from_cart,event_type_view,category_id,price
0,247879837,5879286,0,0,0,1,1487580008800059392,9.51
1,252311454,5700088,0,0,0,1,1487580011677351936,8.76
2,348405118,5848413,0,0,0,1,1487580007675986944,0.79
3,348405118,5862313,1,0,0,0,1487580007675986944,0.79
4,348405118,5884237,0,0,0,1,1487580007675986944,1.75
...,...,...,...,...,...,...,...,...
375,579972058,5820776,1,0,0,0,1487580006317032448,3.97
376,579972058,5820777,1,0,0,0,1487580006317032448,3.97
377,579972058,5870111,1,0,0,0,1487580006317032448,4.29
378,579972652,5896420,0,0,0,1,1487580006300255232,49.21


In [16]:
#grabbing the mean ratings of all users
mean = ratings.groupby(by = 'user_id', as_index = False)['event_type_purchase'].mean()
mean

,user_id,event_type_purchase
0,247879837,0
1,252311454,0
2,348405118,0
3,361564578,0
4,407458156,0
...,...,...
67,579971574,0
68,579971759,0
69,579972058,0
70,579972652,0


In [17]:
products = cosmetic1.groupby(['product_id', 'brand'])['category_id'].count().reset_index()
products = pd.DataFrame(data = products)
products

,product_id,brand,category_id
0,3936,almea,0
1,3936,beautix,0
2,3936,bluesky,0
3,3936,bpw.style,0
4,3936,cnd,0
...,...,...,...
16735,5908747,staleks,0
16736,5908747,strong,0
16737,5908747,uno,0
16738,5908747,yoko,0


In [18]:
products.dtypes

product_id        int64
brand          category
category_id       int64
dtype: object

Getting the brand and product IDs for recommender system in the future.

In [19]:

# pivot table of values
final = pd.pivot_table(cosmetic1, values='event_type_purchase', index=['user_id'],
                    columns=['product_id'])
final

product_id,3936,3978,4497,4554,4600,4607,4640,4645,4689,4751,...,5908210,5908214,5908217,5908219,5908228,5908229,5908236,5908239,5908746,5908747
user_id,,,,,,,,,,,,,,,,,,,,,
247879837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252311454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
348405118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361564578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407458156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579971574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579971759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579972058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#replacing Nan with product average
final_product = final.fillna(final.mean(axis = 0))
final_product

product_id,3936,3978,4497,4554,4600,4607,4640,4645,4689,4751,...,5908210,5908214,5908217,5908219,5908228,5908229,5908236,5908239,5908746,5908747
user_id,,,,,,,,,,,,,,,,,,,,,
247879837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252311454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
348405118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
361564578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
407458156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579971574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579971759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579972058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
#replacing Nan with user average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis =1)
final_user

product_id,3936,3978,4497,4554,4600,4607,4640,4645,4689,4751,...,5908210,5908214,5908217,5908219,5908228,5908229,5908236,5908239,5908746,5908747
user_id,,,,,,,,,,,,,,,,,,,,,
247879837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252311454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
348405118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
361564578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
407458156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579971574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579971759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579972058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
from sklearn.metrics.pairwise import cosine_similarity

# user similarity on replacing NAN by user avg
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

user_id,247879837,252311454,348405118,361564578,407458156,412120092,422556892,426074808,473613801,478193545,...,579971318,579971396,579971401,579971425,579971563,579971574,579971759,579972058,579972652,579973203
user_id,,,,,,,,,,,,,,,,,,,,,
247879837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252311454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
348405118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
361564578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
407458156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
cosine = cosine_similarity(final_product)
np.fill_diagonal(cosine, 0 )
similarity_with_product = pd.DataFrame(cosine,index=final_product.index)
similarity_with_product.columns=final_user.index
similarity_with_product.head()

user_id,247879837,252311454,348405118,361564578,407458156,412120092,422556892,426074808,473613801,478193545,...,579971318,579971396,579971401,579971425,579971563,579971574,579971759,579972058,579972652,579973203
user_id,,,,,,,,,,,,,,,,,,,,,
247879837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252311454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
348405118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
361564578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
407458156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we have two different dataframes with user and product similarity

In [24]:
#finding the most similar neighbors based on a user or product criterion
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['Closest {} Neighbor'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [25]:
# top 10 neighbours for each user
sim_user_10_u = find_n_neighbours(similarity_with_user,10)
sim_user_10_u.head()

,Closest 1 Neighbor,Closest 2 Neighbor,Closest 3 Neighbor,Closest 4 Neighbor,Closest 5 Neighbor,Closest 6 Neighbor,Closest 7 Neighbor,Closest 8 Neighbor,Closest 9 Neighbor,Closest 10 Neighbor
user_id,,,,,,,,,,
247879837,579973203,579972652,509445281,509449396,509636330,513026898,521863546,533289559,537197314,538507783
252311454,579973203,579972652,509445281,509449396,509636330,513026898,521863546,533289559,537197314,538507783
348405118,579973203,579972652,509445281,509449396,509636330,513026898,521863546,533289559,537197314,538507783
361564578,579973203,579972652,509445281,509449396,509636330,513026898,521863546,533289559,537197314,538507783
407458156,579973203,579972652,509445281,509449396,509636330,513026898,521863546,533289559,537197314,538507783


In [26]:
# top 10 neighbours for each user
sim_user_10_p = find_n_neighbours(similarity_with_product,10)
sim_user_10_p.head()

,Closest 1 Neighbor,Closest 2 Neighbor,Closest 3 Neighbor,Closest 4 Neighbor,Closest 5 Neighbor,Closest 6 Neighbor,Closest 7 Neighbor,Closest 8 Neighbor,Closest 9 Neighbor,Closest 10 Neighbor
user_id,,,,,,,,,,
247879837,579973203,579972652,509445281,509449396,509636330,513026898,521863546,533289559,537197314,538507783
252311454,579973203,579972652,509445281,509449396,509636330,513026898,521863546,533289559,537197314,538507783
348405118,579973203,579972652,509445281,509449396,509636330,513026898,521863546,533289559,537197314,538507783
361564578,579973203,579972652,509445281,509449396,509636330,513026898,521863546,533289559,537197314,538507783
407458156,579973203,579972652,509445281,509449396,509636330,513026898,521863546,533289559,537197314,538507783


Getting closest neighbors, chose 10 to begin with so the dataframes do not get cluttered

In [27]:
ratings = ratings.astype({"product_id": str})
shop_user = ratings.groupby(by = 'user_id')['product_id'].apply(lambda x:','.join(x))

In [28]:
def User_item_score1(user):
    product_seen_by_user = final.columns[final[final.index==user].notna().any()].tolist() #creating a dataframe of all items seen by the user
    a = sim_user_10_p[sim_user_10_p.index==user].values #based off of the neighbors we get the values
    b = a.squeeze().tolist()
    d = shop_user[shop_user.index.isin(b)]
    l = ','.join(d.values)
    product_seen_by_similar_users = l.split(',')
    product_under_consideration = list(set(product_seen_by_similar_users)-set(list(map(str, product_seen_by_user))))
    product_under_consideration = list(map(int, product_under_consideration))
    #mapping and listing products for usage
    score = []
    for item in product_under_consideration: #beginning to loop through all the products 
        c = final_product.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = mean.loc[mean['user_id'] == user,'event_type_purchase'].values[0] #getting the users age based on product
        index = f.index.values.squeeze().tolist() #adding to to a list
        corr = similarity_with_product.loc[user,index] 
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1) #using the average score and correlation for a new list
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno) #final score 
        score.append(final_score)
    data = pd.DataFrame({'product_id':product_under_consideration,'score':score}) #creating the dataframe and outputting it
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    product_Name = top_5_recommendation.merge(products, how='inner', on='product_id') #getting the product recommendations
    product_brand = product_Name.product_id.tolist()#   
  
    return product_brand

NameError: name 'top_5_recommendation' is not defined

In [35]:
import warnings
warnings.filterwarnings("ignore")

In [36]:
user = int(input("Enter the user id to whom you want to recommend : ")) #making it so its a bit more interactive
predicted_product = User_item_score1(user)
print(" ")
print("The Recommendations for User Id : ", user)
print("   ")
for i in range(len(predicted_product)):
    if predicted_product[i] != predicted_product[i-1]:
        print(predicted_product[i]) #displaying product recommendations

Enter the user id to whom you want to recommend : 579972652
 
The Recommendations for User Id :  579972652
   
5771114
5732026
34767
5881896
5817138


System displays top 5 products for each user based on previous users.